# Tutorial: Create production ML pipelines with Python SDK v2 (preview) in a Jupyter notebook


**Learning Objectives** - By the end of this two part tutorial, you should be able to use Azure Machine Learning (Azure ML) to productionize your ML project.

This means you will be able to leverage the AzureML Python SDK to:

- connect to your Azure ML workspace
- create Azure ML data assets
- create reusable Azure ML components
- create, validate and run Azure ML pipelines
- deploy the newly-trained model as an endpoint
- call the Azure ML endpoint for inferencing

**Motivations** - This tutorial is intended to introduce Azure ML to data scientists who want to scale up or publish their ML projects. By completing a familiar end-to-end project, which starts by loading the data and ends by creating and calling an online inference endpoint, the user should become familiar with the core concepts of Azure ML and their most common usage. Each step of this tutorial can be modified or performed in other ways that might have security or scalability advantages. We will cover some of those in the Part II of this tutorial, however, we suggest the reader use the provide links in each section to learn more on each topic.

**Requirements** - In order to benefit from this tutorial, you need to have:
- basic understanding of Machine Learning projects workflow
- an Azure subscription. If you don't have an Azure subscription, [create a free account](https://aka.ms/AMLFree) before you begin.
- a working Azure ML workspace. A workspace can be created via Azure Portal, Azure CLI, or Python SDK. [Read more](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?tabs=python).
- a Python environmnet
- [installed Azure Machine Learning Python SDK v2](https://github.com/Azure/azureml-examples/blob/sdk-preview/sdk/setup.sh)

## Introduction

In this tutorial, you'll create an Azure ML pipeline to train a model for credit default prediction. The pipeline handles the data preparation, training and registering the trained model.  You'll then run the pipeline, deploy the model and use it.

The image below shows the pipeline as you'll see it in the AzureML portal once submitted. It's a rather simple pipeline we'll use to walk you through the AzureML SDK v2.

The two steps are first data preparation and second training. 

![Screenshot that shows the AML Pipeline](media/pipeline-overview.jpg "Overview of the pipeline")

An AzureML pipeline that runs from local components, requires several dependent files. Fo better understanding of the project structure, we produce all these dependencies in the notebook cells. By the end of this tutorial, the project structure should look like:
```
e2e-ds-experience  
    components
        data_prep
            data_prep.py
            data_prep.yml  
        train
            train.py
            train.yml   
    dependencies
        conda.yml
    deploy
        sample-request.json
    media
        metrics.jpg
        pipeline-overview.jpg
        user-logs.jpg
    e2e-ml-workflow.ipynb
```
After running this notebook, you should be able to create the project direcetly in the IDE of your choice, instead.

## Set up the pipeline resources

The Azure ML framework can be used from CLI, Python SDK, or studio interface. In this example, you'll use the AzureML Python SDK v2 to create a pipeline. 

Before creating the pipeline, you'll set up the resources the pipeline will use:

* The dataset for training
* The software environment to run the pipeline
* A compute resource to where the job will run

## AML을 시작하기위한 주요흐름

1. Connect to Azure Machine Learning workspace

2. Create compute

3. Create a job environment

4. Build the command job to data preparation and train

5. Configure the Command

6. Submit the job

## Connect to the workspace

Before we dive in the code, you'll need to connect to your Azure ML workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [1]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In the next cell, enter your Subscription ID, Resource Group name and Workspace name. To find your Subscription ID:
1. In the upper right Azure Machine Learning Studio toolbar, select your workspace name.
1. At the bottom, select **View all properties in Azure Portal**
1. Copy the value from Azure Portal into the code.

In [2]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="<subscription_id>",
    resource_group_name="<resource_group_name>",
    workspace_name="<workspace_name>",
)

In [2]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="996049d2-a088-4535-84c6-293862437c2e",
    resource_group_name="rg-aml-demo",
    workspace_name="ssc-mlops",
)

The result is a handler to the workspace that you'll use to manage other resources and jobs.

> [!IMPORTANT]
> Creating MLClient will not connect to the workspace. The client initialization is lazy, it will wait for the first time it needs to make a call (in the notebook below, that will happen during dataset registration).

## Register data from an external url

The data you use for training is usually in one of the locations below:

* Local machine
* Web
* Big Data Storage services (for example, Azure Blob, Azure Data Lake Storage, SQL)
 
Azure ML uses a [`Data`](https://docs.microsoft.com/azure/machine-learning/how-to-create-register-data-assets?tabs=Python-SDK) object to register a reusable definition of data, and consume data within a pipeline. In the section below, you'll consume some data from web url as one example. `Data` assets ets from other sources can be created as well.

## 외부 데이터셋 등록 방법.

In [80]:
# from azure.ai.ml.entities import Data
# from azure.ai.ml.constants import AssetTypes

# web_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"

# credit_data = Data(
#     name="inputdata",
#     path=web_path,
#     type=AssetTypes.URI_FILE,
#     description="Dataset for credit card defaults",
#     tags={"source_type": "web", "source": "UCI ML Repo"},
#     version="1.0.1",
# )

This code just created a `Data` asset, it is ready to be consumed as an input by the pipeline that you'll define in the next sections. In addition, you can register the data to your workspace so it becomes reusable across pipelines.

Registering the data asset will enable you to:

* reuse and share the data asset in future pipelines
* use versions to track the modification to the data asset
* use the data asset from Azure ML designer, which is Azure ML's GUI for pipeline authoring

Since this is the first time that you're making a call to the workspace, you may be asked to authenticate. Once the authentication is complete, you'll then see the dataset registration completion message.

In [81]:
# credit_data = ml_client.data.create_or_update(credit_data)
# print(
#     f"Dataset with name {credit_data.name} was registered to workspace, the dataset version is {credit_data.version}"
# )

## Azure Machine Learning Studio 에서 sscinpudata inputdata를 등록한다.(데이터 메뉴로 이동.)

In [3]:
credit_data = ml_client.data.get(name="sscinpudata", version="1")
print(
    f"Dataset with name {credit_data.name} was registered to workspace, the dataset version is {credit_data.version}"
)

Dataset with name sscinpudata was registered to workspace, the dataset version is 1


In the future, you can fetch the same dataset from the workspace using `credit_dataset = ml_client.data.get("<DATA ASSET NAME>", version='<VERSION>')`.

## Create a Compute Resource to run our pipeline

Each step of an Azure ML pipeline can use a different compute resource for running the specific job of that step. It can be single or multi-node machines with Linux or Windows OS, or a specific compute fabric like Spark.

In this section, we provision a Linux [compute cluster](https://docs.microsoft.com/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python). See the [full list on VM sizes and prices](https://azure.microsoft.com/en-ca/pricing/details/machine-learning/) .

For this tutorial we only need a basic cluster, let's pick a Standard_DS3_v2 model with 2 vCPU cores, 7 GB RAM and create an Azure ML Compute

AML 스튜디오에서 해당 클러스터를 생성할수도 있음.

In [4]:
from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        # Name assigned to the compute cluster
        name="cpu-cluster",
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS3_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.begin_create_or_update(cpu_cluster)

print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)

You already have a cluster named cpu-cluster, we'll reuse it as is.
AMLCompute with name cpu-cluster is created, the compute size is STANDARD_DS3_V2


## Create a job environment for pipeline steps

So far, you've created a development environment on the compute instance, your development machine. You'll also need an [environment](https://docs.microsoft.com/azure/machine-learning/concept-environments) to use for each step of the pipeline. Each step can have its own environment, or you can use some common environments for multiple steps.

In this example, you'll create a conda environment for your jobs, using a conda yaml file.
First, create a directory to store the file in.

In [5]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

Now, create the file in the dependencies directory.

In [6]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0

Overwriting ./dependencies/conda.yml


The specification contains some usual packages, that you'll use in your pipeline (numpy, pip).


Use the *yaml* file to create and register this custom environment in your workspace:

## ML을 위한 환경등록 재사용가능

In [7]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-scikit-learn"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Credit Card Defaults pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="0.1.0",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name aml-scikit-learn is registered to workspace, the environment version is 0.1.0


## Build the training pipeline

Now that you have all assets required to run your pipeline, it's time to build the pipeline itself, using the Azure ML Python SDK v2.

Azure ML pipelines are reusable ML workflows that usually consist of several components. The typical life of a component is:

* Write the yaml specification of the component.
* Optionally, register the component with a name and version in your workspace, to make it reusable and shareable.
* Load that component from the pipeline code.
* Implement the pipeline using this component inputs, outputs and parameters.
* Submit the pipeline.

## Create component 1: data prep 

Let's start by creating the first component. This component handles the preprocessing of the data. The preprocessing task is performed in the *data_prep.py* python file.

First create a source folder for the data_prep component:

In [8]:
import os

data_prep_src_dir = "./components/data_prep"
os.makedirs(data_prep_src_dir, exist_ok=True)

This script performs the simple task of splitting the data into train and test datasets. 

[MLFlow](https://mlflow.org/docs/latest/tracking.html) will be used to log the parameters and metrics during our pipeline run.

## component 1: data_prep.py

In [9]:
%%writefile {data_prep_src_dir}/data_prep.py
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
import logging
import mlflow


def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    args = parser.parse_args()

    # Start Logging
    mlflow.start_run()

    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)

    credit_df = pd.read_excel(args.data, header=1, index_col=0)

    mlflow.log_metric("num_samples", credit_df.shape[0])
    mlflow.log_metric("num_features", credit_df.shape[1] - 1)

    credit_train_df, credit_test_df = train_test_split(
        credit_df,
        test_size=args.test_train_ratio,
    )

    # output paths are mounted as folder, therefore, we are adding a filename to the path
    credit_train_df.to_csv(os.path.join(args.train_data, "data.csv"), index=False)

    credit_test_df.to_csv(os.path.join(args.test_data, "data.csv"), index=False)

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Overwriting ./components/data_prep/data_prep.py


Now that you have a script that can perform the desired task, we can create an Azure ML Component from it. Azure ML support various types of components for performing ML tasks, such as running scripts, data transfer, etc.

A component can be created by calling the component instantiators, or directly writing the defining yaml file. 

You'll use the general purpose **command** that can run command line actions. This command line action can be directly calling system commands or running a script. The inputs/outputs are accessible in the command via the `${{ ... }}` notation. For the second component of this tutorial you will use `yaml` definitions.


In [10]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
data_prep_component = load_component(source=os.path.join(data_prep_src_dir, "data_prep.yml"))

## register the component to the workspace (option)

In [11]:
# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

Component data_prep_credit_defaults with Version 2023-03-07-14-15-31-6104889 is registered


## component 1: command 설정 (programming 설정)

In [89]:
# from azure.ai.ml import command
# from azure.ai.ml import Input, Output

# data_prep_component = command(
#     name="data_prep_credit_defaults",
#     display_name="Data preparation for training",
#     description="reads a .xl input, split the input to train and test",
#     inputs={
#         "data": Input(type="uri_folder"),
#         "test_train_ratio": Input(type="number"),
#     },
#     outputs=dict(
#         train_data=Output(type="uri_folder", mode="rw_mount"),
#         test_data=Output(type="uri_folder", mode="rw_mount"),
#     ),
#     # The source folder of the component
#     code=data_prep_src_dir,
#     command="""python data_prep.py \
#             --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} \
#             --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}} \
#             """,
#     environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
# )

Optionally, register the component in the workspace for future re-use. **command()** is a component builder, in order to fetch the component itself, we need to call the **.component** property from it.

In [90]:
# # Now we register the component to the workspace
# data_prep_component = ml_client.create_or_update(data_prep_component.component)

# # Create (register) the component in your workspace
# print(
#     f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
# )

Component data_prep_credit_defaults with Version 2023-03-07-09-43-39-1746682 is registered


## Create component 2: training (using yaml definition)

The second component that you'll create will consume the training and test data, train a tree based model and return the output model. You'll use Azure ML logging capabilities to record and visualize the learning progress.

You used the `command` class to create your first component. This time you'll use the yaml definition to define the second component. Each method has its own advantages. A yaml definition can actually be checked-in along the code, and would provide a readable history tracking. Also, the same yaml file can be used in the CLI for component deficnition. The programmatic method using `command` can be easier with built-in class documentation and code completion.


Create the directory for this component:

In [11]:
import os

train_src_dir = "./components/train"
os.makedirs(train_src_dir, exist_ok=True)

Create the training script in the directory:

In [12]:
%%writefile {train_src_dir}/train.py
import argparse
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
import os
import pandas as pd
import mlflow


def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])


# Start Logging
mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)


def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    parser.add_argument("--model", type=str, help="path to model file")
    args = parser.parse_args()

    # paths are mounted as folder, therefore, we are selecting the file from folder
    train_df = pd.read_csv(select_first_file(args.train_data))

    # Extracting the label column
    y_train = train_df.pop("default payment next month")

    # convert the dataframe values to array
    X_train = train_df.values

    # paths are mounted as folder, therefore, we are selecting the file from folder
    test_df = pd.read_csv(select_first_file(args.test_data))

    # Extracting the label column
    y_test = test_df.pop("default payment next month")

    # convert the dataframe values to array
    X_test = test_df.values

    print(f"Training with data of shape {X_train.shape}")

    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))

    # # Registering the model to the workspace
    # print("Registering the model via MLFlow")
    # mlflow.sklearn.log_model(
    #     sk_model=clf,
    #     registered_model_name=args.registered_model_name,
    #     artifact_path=args.registered_model_name,
    # )

    # Saving the model to a file
    # mlflow.sklearn.save_model(clf, args.model)
    mlflow.sklearn.save_model(
        sk_model=clf,
        path=os.path.join(args.model, "trained_model"),
    )

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Overwriting ./components/train/train.py


As you can see in this training script, once the model is trained, the model file is saved and registered to the workspace. Now you can use the registered model in inferencing endpoints.


For the environment of this step, you'll use one of the built-in (curated) Azure ML environments. The tag `azureml`, tells the system to use look for the name in curated environments.

First, create the *yaml* file describing the component:

In [13]:
%%writefile {train_src_dir}/train.yml
# <component>
name: train_credit_defaults_model
display_name: Train Credit Defaults Model
# version: 1 # Not specifying a version will automatically update the version
type: command
inputs:
  train_data: 
    type: uri_folder
  test_data: 
    type: uri_folder
  learning_rate:
    type: number     
  registered_model_name:
    type: string
outputs:
  model:
    type: uri_folder
code: .
environment:
  # for this step, we'll use an AzureML curate environment
  azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1
command: >-
  python train.py 
  --train_data ${{inputs.train_data}} 
  --test_data ${{inputs.test_data}} 
  --learning_rate ${{inputs.learning_rate}}
  --registered_model_name ${{inputs.registered_model_name}} 
  --model ${{outputs.model}}
# </component>


Overwriting ./components/train/train.yml


Once the `yaml` file and the script are ready, you can create your component using `load_component()`. 

In [14]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source=os.path.join(train_src_dir, "train.yml"))

## now create and register the component:(option)

In [16]:
# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

Component train_credit_defaults_model with Version 2023-03-07-14-15-54-0513027 is registered


## Create component 3: Scoring (using yaml definition)


In [15]:
import os

score_src_dir = "./components/score"
os.makedirs(score_src_dir, exist_ok=True)

In [18]:
%%writefile {score_src_dir}/score.py
import argparse
from pathlib import Path
from sklearn.metrics import classification_report
import os
import mlflow

parser = argparse.ArgumentParser("score")
parser.add_argument("--model_input", type=str, help="Path of input model")
parser.add_argument("--test_data", type=str, help="Path to test data")
parser.add_argument("--score_output", type=str, help="Path of scoring output")

args = parser.parse_args()

print("hello scoring world...")

lines = [
    f"Model path: {args.model_input}",
    f"Test data path: {args.test_data}",
    f"Scoring output path: {args.score_output}",
]

for line in lines:
    print(line)

# Load the test data with predicted values

print("mounted_path files: ")
arr = os.listdir(args.test_data)

print(arr)
df_list = []
for filename in arr:
    print("reading file: %s ..." % filename)
    with open(os.path.join(args.predictions, filename), "r") as handle:
        # print (handle.read())
        input_df = pd.read_csv((Path(args.predictions) / filename))
        df_list.append(input_df)

test_data = df_list[0]

# Load the model from input port
pathmodel = os.path.join(args.model_input, "trained_model")
model = mlflow.sklearn.load_model(pathmodel)

actuals = test_data["actual_cost"]
predictions = test_data["predicted_cost"]

# Print the results of scoring the predictions against actual values in the test data
# The coefficients
# print("Coefficients: \n", model)

# # Load the model from input port
# # Here only print the model as text since it is a dummy one
# model = (Path(args.model_input) / "model.txt").read_text()
# print("Model: ", model)

# # Do scoring with the input model
# # Here only print text to output file as demo
(Path(args.score_output) / "score.txt").write_text(
    "Scored with the following mode:\n{}".format(model)
)


Overwriting ./components/score/score.py


In [19]:
%%writefile {score_src_dir}/score.yml
type: command

name: score_data
display_name: Score Data
description: A dummy scoring component

inputs:
  model_input:
    type: uri_folder
  test_data:
    type: uri_folder
outputs:
  score_output:
    type: uri_folder
environment: azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1
code: .
command: >-
  python score.py 
  --model_input ${{inputs.model_input}} 
  --test_data ${{inputs.test_data}}
  --score_output ${{outputs.score_output}}


Overwriting ./components/score/score.yml


In [20]:
# importing the Component Package
from azure.ai.ml import load_component
# Loading the component from the yml file
score_component = load_component(source=os.path.join(score_src_dir, "score.yml"))

In [21]:
# Now we register the component to the workspace
score_component = ml_client.create_or_update(score_component)

# Create (register) the component in your workspace
print(
    f"Component {score_component.name} with Version {score_component.version} is registered"
)

Uploading score (0.0 MBs): 100%|██████████| 2041/2041 [00:00<00:00, 65580.68it/s]




Component score_data with Version 2023-03-07-14-16-12-0679943 is registered


## Create the pipeline from components

Now that both your components are defined and registered, you can start implementing the pipeline.

Here, you'll use *input data*, *split ratio* and *registered model name* as input variables. Then call the components and connect them via their inputs /outputs identifiers. The outputs of each step can be accessed via the `.outputs` property.

The python functions returned by `load_component()` work as any regular python function that we'll use within a pipeline to call each step.

To code the pipeline, we use a specific `@dsl.pipeline` decorator that identifies the Azure ML pipelines. In the decorator, we can specify the pipeline description and default resources like compute and storage. Like a python function, pipelines can have inputs, you can then create multiple instances of a single pipeline with different inputs.

Here, we used *input data*, *split ratio* and *registered model name* as input variables. We then call the components and connect them via their inputs /outputs identifiers. The outputs of each step can be accessed via the `.outputs` property.

In [22]:
# the dsl decorator tells the sdk that we are defining an Azure ML pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute=cpu_compute_target,
    description="E2E data_perp-train pipeline",
)
def credit_defaults_pipeline(
    pipeline_job_data_input,
    pipeline_job_test_train_ratio,
    pipeline_job_learning_rate,
    pipeline_job_registered_model_name,
):
    # using data_prep_function like a python call with its own inputs
    data_prep_job = data_prep_component(
        data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,
    )

    # using train_func like a python call with its own inputs
    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        learning_rate=pipeline_job_learning_rate,  # note: using a pipeline input as parameter
        registered_model_name=pipeline_job_registered_model_name,
    )

    score_job = score_component(
        model_input=train_job.outputs.model,
        test_data=data_prep_job.outputs.test_data,
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
        "pipeline_job_test_data": data_prep_job.outputs.test_data,
        "pipeline_job_model": train_job.outputs.model,
        "pipeline_job_score": score_job.outputs.score_output,
    }

Now use your pipeline definition to instantiate a pipeline with your dataset, split rate of choice and the name you picked for your model.

In [23]:
registered_model_name = "credit_defaults_model"

# Let's instantiate the pipeline with the parameters of our choice
pipeline = credit_defaults_pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=credit_data.path),
    pipeline_job_test_train_ratio=0.25,
    pipeline_job_learning_rate=0.05,
    pipeline_job_registered_model_name=registered_model_name,
)

## Submit the job 

It's now time to submit the job to run in Azure ML. This time you'll use `create_or_update`  on `ml_client.jobs`.

Here you'll also pass an experiment name. An experiment is a container for all the iterations one does on a certain project. All the jobs submitted under the same experiment name would be listed next to each other in Azure ML studio.

Once completed, the pipeline will register a model in your workspace as a result of training.

In [24]:
import webbrowser

# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="e2e_registered_components",
)
# open the pipeline in web browser
webbrowser.open(pipeline_job.studio_url)

False

In [25]:
ml_client.jobs.stream(pipeline_job.name)

RunId: mighty_date_xg1vvcy7rq
Web View: https://ml.azure.com/runs/mighty_date_xg1vvcy7rq?wsid=/subscriptions/996049d2-a088-4535-84c6-293862437c2e/resourcegroups/rg-aml-demo/workspaces/ssc-mlops

Streaming logs/azureml/executionlogs.txt

[2023-03-07 14:16:28Z] Submitting 1 runs, first five are: f2b95b55:99568ea3-ac36-479c-8280-293d156d5e65
[2023-03-07 14:16:30Z] Completing processing run id 99568ea3-ac36-479c-8280-293d156d5e65.
[2023-03-07 14:16:30Z] Submitting 1 runs, first five are: 0b9be8ce:968044c7-a479-483e-9d43-7d3f6d5fc931
[2023-03-07 14:16:31Z] Completing processing run id 968044c7-a479-483e-9d43-7d3f6d5fc931.
[2023-03-07 14:16:32Z] Submitting 1 runs, first five are: d6588ed4:92204c12-0bc2-4714-b103-774e245701ea


You can track the progress of your pipeline, by using the link generated in the cell above or in this notebook using the following code:
```python
    ml_client.jobs.stream(pipeline_job.name)
```

When you select on each component, you'll see more information about the results of that component. 
There are two important parts to look for at this stage:
* `Outputs+logs` > `user_logs` > `std_log.txt`
This section shows the script run sdtout.

* `Outputs+logs` > `Metric`
This section shows different logged metrics. In this example. mlflow `autologging`, has automatically logged the training metrics.

## 

## e2e_registered_component 실험 리스트 조회

In [100]:
import mlflow

exp = mlflow.get_experiment_by_name("e2e_registered_components")
exp

<Experiment: artifact_location='', creation_time=1678064718535, experiment_id='589ba941-0fdf-41f1-8a44-39204466f49a', last_update_time=None, lifecycle_stage='active', name='e2e_registered_components', tags={}>

In [101]:
run_list = mlflow.search_runs(exp.experiment_id)
run_list

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.num_samples,metrics.num_features,metrics.training_f1_score,metrics.training_roc_auc_score,...,params.min_impurity_decrease,params.n_estimators,params.ccp_alpha,tags.mlflow.user,tags.mlflow.rootRunId,tags.mlflow.parentRunId,tags.azureml.nodeid,tags.azureml.pipeline,tags.estimator_name,tags.estimator_class
0,clever_cloud_vjn3fbrmhj,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-06 01:05:19.789000+00:00,2023-03-06 01:10:00.856000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,김상우,clever_cloud_vjn3fbrmhj,None,None,None,None,None
1,d7ca41fa-587d-450a-a751-b37a68e80488,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-06 01:07:35.653000+00:00,2023-03-06 01:08:36.840000+00:00,30000.0,23.0,NaN,NaN,...,None,None,None,김상우,clever_cloud_vjn3fbrmhj,clever_cloud_vjn3fbrmhj,5751f057,clever_cloud_vjn3fbrmhj,None,None
2,56ff461b-a59f-470d-89e7-79a258edef6b,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-06 01:08:47.631000+00:00,2023-03-06 01:09:59.371000+00:00,NaN,NaN,0.803281,0.795926,...,0.0,100,0.0,김상우,clever_cloud_vjn3fbrmhj,clever_cloud_vjn3fbrmhj,f5e3994d,clever_cloud_vjn3fbrmhj,GradientBoostingClassifier,sklearn.ensemble._gb.GradientBoostingClassifier
3,silly_beach_8px3xdh5hc,589ba941-0fdf-41f1-8a44-39204466f49a,FAILED,,2023-03-07 00:57:43.219000+00:00,2023-03-07 01:00:29.546000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,김상우,silly_beach_8px3xdh5hc,None,None,None,None,None
4,b5f4f39c-6245-458a-870c-e93fe3bb9337,589ba941-0fdf-41f1-8a44-39204466f49a,FAILED,,2023-03-07 00:59:40.244000+00:00,2023-03-07 01:00:27.534000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,김상우,silly_beach_8px3xdh5hc,silly_beach_8px3xdh5hc,67989775,silly_beach_8px3xdh5hc,None,None
5,mango_wire_1bvhvh3w8m,589ba941-0fdf-41f1-8a44-39204466f49a,FAILED,,2023-03-07 01:09:35.974000+00:00,2023-03-07 01:12:45.431000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,김상우,mango_wire_1bvhvh3w8m,None,None,None,None,None
6,25fdf402-d68b-4efe-9e80-77413d124598,589ba941-0fdf-41f1-8a44-39204466f49a,FAILED,,2023-03-07 01:11:55.134000+00:00,2023-03-07 01:12:44.179000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,김상우,mango_wire_1bvhvh3w8m,mango_wire_1bvhvh3w8m,cfb683cb,mango_wire_1bvhvh3w8m,None,None
7,willing_house_7kpklq0km7,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-07 01:35:37.731000+00:00,2023-03-07 01:39:44.311000+00:00,NaN,NaN,NaN,NaN,...,None,None,None,김상우,willing_house_7kpklq0km7,None,None,None,None,None
8,703759d3-2c81-45ba-83ff-d7942fca0003,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-07 01:37:34.542000+00:00,2023-03-07 01:38:27.963000+00:00,30000.0,23.0,NaN,NaN,...,None,None,None,김상우,willing_house_7kpklq0km7,willing_house_7kpklq0km7,41e31f72,willing_house_7kpklq0km7,None,None
9,8a29575c-7838-4400-b300-1109f5b10a25,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-07 01:38:38.841000+00:00,2023-03-07 01:39:43.024000+00:00,NaN,NaN,0.805423,0.793803,...,0.0,100,0.0,김상우,willing_house_7kpklq0km7,willing_house_7kpklq0km7,72679786,willing_house_7kpklq0km7,GradientBoostingClassifier,sklearn.ensemble._gb.GradientBoostingClassifier


In [102]:
# Get the run with the highest training R2 score
run_list = mlflow.search_runs(exp.experiment_id)
best_run_result = run_list.fillna(0)

## Experiment list 중에서 비교를 원하는 Metric을 선택해서 가장 좋은 값을 가진 experiment id 를 추출한다.
아래는 metrics.training_f1_score 값을 기준으로 비교한다.

In [103]:
best_run_result = best_run_result.sort_values('metrics.training_f1_score', ascending=False)
best_run_result

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.num_samples,metrics.num_features,metrics.training_f1_score,metrics.training_roc_auc_score,...,params.min_impurity_decrease,params.n_estimators,params.ccp_alpha,tags.mlflow.user,tags.mlflow.rootRunId,tags.mlflow.parentRunId,tags.azureml.nodeid,tags.azureml.pipeline,tags.estimator_name,tags.estimator_class
9,8a29575c-7838-4400-b300-1109f5b10a25,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-07 01:38:38.841000+00:00,2023-03-07 01:39:43.024000+00:00,0.0,0.0,0.805423,0.793803,...,0.0,100,0.0,김상우,willing_house_7kpklq0km7,willing_house_7kpklq0km7,72679786,willing_house_7kpklq0km7,GradientBoostingClassifier,sklearn.ensemble._gb.GradientBoostingClassifier
12,874b1810-0d0f-4d76-8ffa-770ac53b6c6d,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-07 09:43:48.825000+00:00,2023-03-07 09:43:48.851000+00:00,0.0,0.0,0.805423,0.793803,...,0.0,100,0.0,김상우,serene_ocean_m32ymk9k4n,serene_ocean_m32ymk9k4n,c5d10fa7,serene_ocean_m32ymk9k4n,GradientBoostingClassifier,sklearn.ensemble._gb.GradientBoostingClassifier
2,56ff461b-a59f-470d-89e7-79a258edef6b,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-06 01:08:47.631000+00:00,2023-03-06 01:09:59.371000+00:00,0.0,0.0,0.803281,0.795926,...,0.0,100,0.0,김상우,clever_cloud_vjn3fbrmhj,clever_cloud_vjn3fbrmhj,f5e3994d,clever_cloud_vjn3fbrmhj,GradientBoostingClassifier,sklearn.ensemble._gb.GradientBoostingClassifier
0,clever_cloud_vjn3fbrmhj,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-06 01:05:19.789000+00:00,2023-03-06 01:10:00.856000+00:00,0.0,0.0,0.000000,0.000000,...,0,0,0,김상우,clever_cloud_vjn3fbrmhj,0,0,0,0,0
1,d7ca41fa-587d-450a-a751-b37a68e80488,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-06 01:07:35.653000+00:00,2023-03-06 01:08:36.840000+00:00,30000.0,23.0,0.000000,0.000000,...,0,0,0,김상우,clever_cloud_vjn3fbrmhj,clever_cloud_vjn3fbrmhj,5751f057,clever_cloud_vjn3fbrmhj,0,0
3,silly_beach_8px3xdh5hc,589ba941-0fdf-41f1-8a44-39204466f49a,FAILED,,2023-03-07 00:57:43.219000+00:00,2023-03-07 01:00:29.546000+00:00,0.0,0.0,0.000000,0.000000,...,0,0,0,김상우,silly_beach_8px3xdh5hc,0,0,0,0,0
4,b5f4f39c-6245-458a-870c-e93fe3bb9337,589ba941-0fdf-41f1-8a44-39204466f49a,FAILED,,2023-03-07 00:59:40.244000+00:00,2023-03-07 01:00:27.534000+00:00,0.0,0.0,0.000000,0.000000,...,0,0,0,김상우,silly_beach_8px3xdh5hc,silly_beach_8px3xdh5hc,67989775,silly_beach_8px3xdh5hc,0,0
5,mango_wire_1bvhvh3w8m,589ba941-0fdf-41f1-8a44-39204466f49a,FAILED,,2023-03-07 01:09:35.974000+00:00,2023-03-07 01:12:45.431000+00:00,0.0,0.0,0.000000,0.000000,...,0,0,0,김상우,mango_wire_1bvhvh3w8m,0,0,0,0,0
6,25fdf402-d68b-4efe-9e80-77413d124598,589ba941-0fdf-41f1-8a44-39204466f49a,FAILED,,2023-03-07 01:11:55.134000+00:00,2023-03-07 01:12:44.179000+00:00,0.0,0.0,0.000000,0.000000,...,0,0,0,김상우,mango_wire_1bvhvh3w8m,mango_wire_1bvhvh3w8m,cfb683cb,mango_wire_1bvhvh3w8m,0,0
7,willing_house_7kpklq0km7,589ba941-0fdf-41f1-8a44-39204466f49a,FINISHED,,2023-03-07 01:35:37.731000+00:00,2023-03-07 01:39:44.311000+00:00,0.0,0.0,0.000000,0.000000,...,0,0,0,김상우,willing_house_7kpklq0km7,0,0,0,0,0


In [104]:
# Print the best run ID and score
if len(best_run_result) > 0:
    best_run_id = best_run_result.iloc[0]['run_id']
    best_score = best_run_result.iloc[0]['metrics.training_f1_score']    
    print(best_run_id, best_score)
else:
    print("No runs found in the experiment.")

8a29575c-7838-4400-b300-1109f5b10a25 0.8054226894119054


## Deploy the model as an online endpoint (이하는 옵션)

Now deploy your machine learning model as a web service in the Azure cloud, an [`online endpoint`](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).

To deploy a machine learning service, you usually need:

* The model assets (filed, metadata) that you want to deploy. You've already registered these assets in your training component.
* Some code to run as a service. The code executes the model on a given input request. This entry script receives data submitted to a deployed web service and passes it to the model, then returns the model's response to the client. The script is specific to your model. The entry script must understand the data that the model expects and returns. When using a MLFlow model, as in this tutorial, this script is automatically created for you. Samples of scoring scripts can be found [here](https://github.com/Azure/azureml-examples/tree/sdk-preview/sdk/endpoints/online).



## Create a new online endpoint

Now that you have a registered model and an inference script, it's time to create your online endpoint. The endpoint name needs to be unique in the entire Azure region. For this tutorial, you'll create a unique name using [`UUID`](https://en.wikipedia.org/wiki/Universally_unique_identifier#:~:text=A%20universally%20unique%20identifier%20(UUID,%2C%20for%20practical%20purposes%2C%20unique.).

In [105]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

In [106]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint_result = ml_client.begin_create_or_update(endpoint).result()

print(
    f"Endpint {endpoint_result.name} provisioning state: {endpoint_result.provisioning_state}"
)

Once you've created an endpoint, you can retrieve it as below:

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

## Deploy the model to the endpoint

Once the endpoint is created, deploy the model with the entry script. Each endpoint can have multiple deployments and direct traffic to these deployments can be specified using rules. Here you'll create a single deployment that handles 100% of the incoming traffic. We have chosen a color name for the deployment, for example, *blue*, *green*, *red* deployments, which is arbitrary.

You can check the *Models* page on the Azure ML studio, to identify the latest version of your registered model. Alternatively, the code below will retrieve the latest version number for you to use.

In [ ]:
# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]

)
latest_model_version

In [ ]:
for m in ml_client.models.list(name=registered_model_name):
    print(m.flavors)

Deploy the latest version of the model.  

> [!NOTE]
> Expect this deployment to take approximately 6 to 8 minutes.

In [ ]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)


# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

blue_deployment_results = ml_client.online_deployments.begin_create_or_update(
    blue_deployment
).result()

print(
    f"Deployment {blue_deployment_results.name} provisioning state: {blue_deployment_results.provisioning_state}"
)

### Test with a sample query

Now that the model is deployed to the endpoint, you can run inference with it.

Create a sample request file following the design expected in the run method in the score script.

In [ ]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [ ]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
  }
}

In [ ]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

## Clean up resources

If you're not going to use the endpoint, delete it to stop using the resource.  Make sure no other deployments are using an endpoint before you delete it.


> [!NOTE]
> Expect this step to take approximately 6 to 8 minutes.

In [ ]:
# ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

## Next Steps

Learn more about [Azure ML logging](https://github.com/Azure/azureml-examples/blob/sdk-preview/notebooks/mlflow/mlflow-v1-comparison.ipynb).